## Google Assistant 

This is the code you used earlier to run the Google Assistant on the AIY Voice Kit. 

In [ ]:
import argparse
import locale
import logging
import signal
import sys
import time

from aiy.assistant.grpc import AssistantServiceClientWithLed
from aiy.assistant import auth_helpers, device_helpers, device_handler_helpers
from aiy.board import Board
from aiy.leds import Leds, Color

#### Setup

Here we set the volume (0-100), language and create a logger object allowing all the different parts of the code to write messages to a log - or here to the screen.

In [ ]:
logger = logging.getLogger(__name__)
language, _ = locale.getdefaultlocale()
volume = 10

logging.basicConfig(level=logging.DEBUG)
signal.signal(signal.SIGTERM, lambda signum, frame: sys.exit(0))

credentials = auth_helpers.get_assistant_credentials()
device_model_id, device_id = device_helpers.get_ids_for_service(credentials)
device_handler = device_handler_helpers.DeviceRequestHandler(device_id)

logger.info('device_model_id: %s', device_model_id)
logger.info('device_id: %s', device_id)

#### Custom handlers

Once we have configured curom traits (like turning the servo), we need to add the code that handles those requests here.

In [ ]:
#------------------------------------------------------------------------------
# Add custom device handlers here
#------------------------------------------------------------------------------

## None for now

#------------------------------------------------------------------------------    

#### Assistant loop

Here is the main part. Basically it waits for the user to press the button and then calls the assistent.conversation() routine to deal with one convsersation. Once it send it goes back waiting for someone to press the button and so on.

In [ ]:
with Board() as board:
    assistant = AssistantServiceClientWithLed(board=board,
                                              volume_percentage=volume,
                                              language_code=language,
                                              credentials=credentials,
                                              device_model_id=device_model_id,
                                              device_id=device_id,
                                              device_handler=device_handler)

    try:
        while True:
            logging.info('Press button to start conversation...')
            board.button.wait_for_press()
            logging.info('Conversation started!')
            assistant.conversation()
    except KeyboardInterrupt:
        print("Bye")